Load ball_by_ball data into dataframe

Load player_match data to dataframe


Using Python SQL to calculate Bowler level stats for each season.

GOAL: Create stats report for bowler level start something which looks in this link https://www.iplt20.com/stats/2008

we have to calculate stats at SEASON level & INNINGS/MATCH level.

we will need to do some Joins, aggregations,window operations & create intermediate data frames to achive the required columns and values

Final reports will be written to dbfs:/FileStore/ipl_data/gold/bowlers path in gold zone


In [0]:
### Create Temporary views to avoid recalculations #####
### INNS_STATS 
### BBI_WICKETS
### dot_balls
### over_ball_count
### bowler_played_matches
### bowler_over_order_wicket_cnt
### bowler_current_over_next_over
### bowler_cont_wick_taken_overs
### wicket_taken_balls
### overs_with_contunious_wickets
### HATRICK


In [0]:

INNS_STATS="""select * from (
select 
season ,
player_id,
player_name,
player_team,
count( match_id) as MAT,
sum( inn) as INNS,
sum(overs) as OV,
coalesce(sum(runs),0) as RUNS,
sum(wickets) as WKTS,
coalesce(round(NULLIF(sum(runs),0)/NULLIF(sum(wickets),0),2),0) as AVG,
coalesce(round(NULLIF(sum(runs),0)/NULLIF(sum(overs),0),2),0) as ECON,
coalesce(round(NULLIF(sum(overs)*6,0)/NULLIF(sum(wickets),0),2),0) as SR,
sum(four_wick) as `4W`,
sum(five_wick) as `5W`
from
(
select a.season_year as season,
	   a.match_id,
	   a.player_id,
	   a.player_name,
	   a.player_team_abbrivation as player_team,
	   count(distinct b.bowler) as inn,
	   count(distinct b.over_id) as overs,
	   sum(b.runs_scored+b.bowler_extras)  as runs,
	   sum(case when b.bowler_wicket=1 then 1 else 0 end) as wickets,
	   case when sum(bowler_wicket)=4 then 1 else 0 end as four_wick,
	   case when sum(bowler_wicket)>=5 then 1 else 0 end as five_wick
from player_match a
left join 
ball_by_ball_stats b
on a.match_id=b.match_id and a.season_year=b.season and a.player_id=b.bowler
GROUP BY 1,2,3,4,5
) a
group by 1,2,3,4) a 
where a.WKTS>0"""


BBI_WICKETS="""select bowler,wickets,runs from 
(
select 
        match_id,
        bowler, 
		season,
		sum(bowler_wicket) as wickets,
	    sum(runs_scored+bowler_extras)  as runs,
		dense_rank() over(partition by bowler order by sum(bowler_wicket) desc, sum(runs_scored+bowler_extras) asc )  as bbi_rnk,
		sum(runs_scored) as runs_concided 
		from ball_by_ball_stats   
	    group by 1,2,3 ) a 
where bbi_rnk=1
group by 1,2,3"""


dot_balls=""" 
select 
	season,
	match_id,
	bowler,
	count(*) as DOTS
from 
	ball_by_ball_stats a
join 
	players_data b on a.bowler=b.player_id 
where runs_scored+extra_runs+bowler_extras+extra_runs=0
group by 1,2,3"""

# at over level 
over_ball_count= """ 
select 
season,
match_id,
bowler,
over_id,
sum(case when bowler_extras=0 then 1 else 0 end) as ball_count,  --balls bowled ignoring extra balls bowled by bowler
sum(case when bowler_wicket=1 then 1 else 0 end) as over_wickets,  -- wickets taken in each over
sum(runs_scored+bowler_extras) as runs  --runs conceded in each over
from ball_by_ball_stats 
group by 1,2,3,4"""



spark.sql(INNS_STATS).createOrReplaceTempView('INNS_STATS')

spark.sql(BBI_WICKETS).createOrReplaceTempView('BBI_WICKETS')

spark.sql(dot_balls).createOrReplaceTempView('dot_balls')

spark.sql(over_ball_count).createOrReplaceTempView('over_ball_count')


In [0]:
##purple Cap

gold_path = dbutils.widgets.get("gold_path")

PURPLE_CAP="""select  
a.SEASON as season_year,
a.PLAYER_ID,
a.PLAYER_NAME,
a.PLAYER_TEAM,
a.WKTS,
a.MAT,
a.INNS,
a.OV,
a.RUNS,
concat(b.wickets,'/',b.runs) as BBI,
a.AVG,
a.ECON,
a.SR,
a.`4W`,
a.`5W`
from INNS_STATS a
join 
BBI_WICKETS b on a.PLAYER_ID=b.bowler
Order by season asc, wkts desc;"""


spark.sql(f"""
    CREATE OR REPLACE TABLE PURPLE_CAP
    USING DELTA
    PARTITIONED BY (season_year)
    LOCATION "{gold_path}/purple_cap"
    AS {PURPLE_CAP}
""")



Most_Maidens="""
select  
a.SEASON as season_year,
a.PLAYER_ID,
a.PLAYER_NAME,
a.PLAYER_TEAM,
b.MAID,
a.MAT,
a.INNS,
a.OV,
a.RUNS,
a.WKTS,
a.AVG,
a.ECON,
a.SR,
a.`4W`,
a.`5W`
from INNS_STATS a
join 
(select season,bowler,b.player_name,count(*) as MAID
from (
select season,match_id,bowler,over_id,
sum(runs_scored+bowler_extras) as runs,
count(ball_id) as ball_count
from ball_by_ball_stats 
group by 1,2,3,4 
having sum(runs_scored+bowler_extras)=0 and count(ball_id) >=6 ) a
join 
players_data b on a.bowler=b.player_id
group by 1,2,3)  b  on a.season=b.season and a.player_id=b.bowler 
order by season_year asc, MAID desc"""

spark.sql(f"""
    CREATE OR REPLACE TABLE Most_Maidens
    USING DELTA
    PARTITIONED BY (season_year)
    LOCATION "{gold_path}/Most_Maidens"
    AS {Most_Maidens}
""")



Most_dots_balls="""
select  
a.SEASON as season_year,
a.PLAYER_ID,
a.PLAYER_NAME,
a.PLAYER_TEAM,
b.DOTS,
a.MAT,
a.INNS,
a.OV,
a.RUNS,
a.WKTS,
a.AVG,
a.ECON,
a.SR,
a.`4W`,
a.`5W`
from INNS_STATS a
join
(select season,bowler,sum(DOTS) as DOTS
from dot_balls a
group by 1,2) b on a.season=b.season and a.player_id=b.bowler
order by b.DOTS desc"""

spark.sql(f"""
    CREATE OR REPLACE TABLE Most_dots_balls
    USING DELTA
    PARTITIONED BY (season_year)
    LOCATION "{gold_path}/most_dots_balls"
    AS {Most_dots_balls}
""")

Best_Bowling_Average="""
select  
season_year,
PLAYER_ID,
PLAYER_NAME,
PLAYER_TEAM,
AVG,
MAT,
INNS,
OV,
RUNS,
WKTS,
BBI,
ECON,
SR,
`4W`,
`5W`
from PURPLE_CAP 
Order by season_year asc, AVG  asc;"""

spark.sql(f"""
    CREATE OR REPLACE TABLE Best_Bowling_Average
    USING DELTA
    PARTITIONED BY (season_year)
    LOCATION "{gold_path}/best_bowling_average"
    AS {Best_Bowling_Average}
""")


Best_Bowling_Economy="""
select  
season_year,
PLAYER_ID,
PLAYER_NAME,
PLAYER_TEAM,
ECON,
MAT,
INNS,
OV,
RUNS,
WKTS,
BBI,
AVG,
SR,
`4W`,
`5W`
from PURPLE_CAP 
Order by season_year asc, ECON  asc;"""

spark.sql(f"""
    CREATE OR REPLACE TABLE Best_Bowling_Economy
    USING DELTA
    PARTITIONED BY (season_year)
    LOCATION "{gold_path}/best_bowling_economy"
    AS {Best_Bowling_Economy}
""")

Best_Bowling_Strike_Rate="""
select  
season_year,
PLAYER_ID,
PLAYER_NAME,
PLAYER_TEAM,
SR,
MAT,
INNS,
OV,
RUNS,
WKTS,
BBI,
AVG,
ECON,
`4W`,
`5W`
from PURPLE_CAP 
Order by season_year asc, SR  asc;"""

spark.sql(f"""
    CREATE OR REPLACE TABLE Best_Bowling_Strike_Rate
    USING DELTA
    PARTITIONED BY (season_year)
    LOCATION "{gold_path}/best_bowling_strike_rate"
    AS {Best_Bowling_Strike_Rate}
""")



Out[4]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
describe formatted PURPLE_CAP;

col_name,data_type,comment
season_year,int,null
PLAYER_ID,int,null
PLAYER_NAME,string,null
PLAYER_TEAM,string,null
WKTS,bigint,null
MAT,bigint,null
INNS,bigint,null
OV,bigint,null
RUNS,bigint,null
BBI,string,null


Innings/Match level reports
### Stats At innings/match level ##
1) Most Dot Balls (innings) 
2) Best_Bowling_Strike_Rate (innings)
3) Best_Bowling_Strike_Rate  (innings)
4) Most_Runs_Conceded (innings)
5) Best_Bowling_Figures_innings

In [0]:
##creaate a temp table 
Best_Bowling_Figures_innings_stg="""
select a.season as season_year,
a.match_id,
a.bowler,
d.player_name,
d.player_team_abbrivation as player_team,
case when ball_count IS NULL then cast(a.over_count as VARCHAR(5)) else concat(a.over_count,'.',b.ball_count) end as OV,
c.runs,
c.wkts,
concat(c.wkts,'/',c.runs) as BBI,
coalesce(round(NULLIF(total_match_balls,0)/NULLIF(wkts,0),2),0) as SR,
d.opposit_team_abbrivation as AGNAIST,
e.venue_name as venue,
e.match_date
from 
(select season,match_id,bowler,
 sum(ball_count) as total_match_balls,
sum(case when ball_count>=6 then 1 else 0 end) as  over_count
from over_ball_count
group by 1,2,3 ) a
left join 
(select season,match_id,bowler,ball_count
from over_ball_count where ball_count<6) b on a.season=b.season and a.match_id=b.match_id and a.bowler=b.bowler
left join 
(select season,match_id,bowler,sum(runs) as runs,sum(over_wickets) as wkts from over_ball_count group by 1,2,3) c
on a.season=c.season and a.match_id=c.match_id and a.bowler=c.bowler
left join 
player_match d on a.bowler=d.player_id and a.season=d.season_year and a.match_id=d.match_id 
left join 
matches e on a.season=e.season_year and a.match_id=e.match_id """

spark.sql(Best_Bowling_Figures_innings_stg).createOrReplaceTempView('Best_Bowling_Figures_innings_stg')

bowler_played_matches =""" 
select 
	season ,match_id,bowler
from ball_by_ball_stats a 
group by 1,2,3"""

spark.sql(bowler_played_matches).createOrReplaceTempView('bowler_played_matches')




In [0]:

Best_Bowling_Figures_innings="""	
select  season_year,
	bowler as player_id,
	player_name,
	player_team,
	BBI,
	OV,
	RUNS,
	WKTS,
	SR,
	AGNAIST, 
	venue,
	match_date
from Best_Bowling_Figures_innings_stg order by season_year asc, wkts desc;"""

spark.sql(f"""
    CREATE OR REPLACE TABLE Best_Bowling_Figures_innings
    USING DELTA
    PARTITIONED BY (season_year)
    LOCATION "{gold_path}/Best_Bowling_Figures_innings"
    AS {Best_Bowling_Figures_innings}
""")

Most_Dot_Balls_Innings="""select 
a.season as season_year,
a.bowler as player_id,
b.player_name,
b.player_team,
a.dots,
b.ov,
b.runs,
b.wkts,
b.sr,
b.AGNAIST, 
b.venue,
b.match_date
from dot_balls a
left join 
Best_Bowling_Figures_innings_stg b on a.match_id=b.match_id and a.season=b.season_year and a.bowler=b.bowler
order by a.season asc , dots desc"""


spark.sql(f"""
    CREATE OR REPLACE TABLE Most_Dot_Balls_Innings
    USING DELTA
    PARTITIONED BY (season_year)
    LOCATION "{gold_path}/Most_Dot_Balls_Innings"
    AS {Most_Dot_Balls_Innings}
""")

Best_Bowling_Economy_Innings="""select 
a.season as season_year,
a.bowler as player_id,
b.player_name,
b.player_team,
coalesce(round(NULLIF(b.runs,0)/NULLIF(cast(b.ov as decimal),0),2),0) as ECON,
b.ov,
b.runs,
b.wkts,
a.dots,
b.sr,
b.AGNAIST, 
b.venue,
b.match_date
from dot_balls a
left join 
Best_Bowling_Figures_innings_stg b on a.match_id=b.match_id and a.season=b.season_year and a.bowler=b.bowler
where cast(b.ov as decimal)>1
order by a.season asc , ECON asc"""

spark.sql(f"""
    CREATE OR REPLACE TABLE Best_Bowling_Economy_Innings
    USING DELTA
    PARTITIONED BY (season_year)
    LOCATION "{gold_path}/Best_Bowling_Economy_Innings"
    AS {Best_Bowling_Economy_Innings}
""")


Best_Bowling_Strike_Rate_Innings="""	
select 
a.season as season_year,
a.bowler as player_id,
b.player_name,
b.player_team,
b.sr,
b.ov,
b.runs,
b.wkts,
b.AGNAIST, 
b.venue,
b.match_date
from bowler_played_matches a
left join 
Best_Bowling_Figures_innings_stg b on a.match_id=b.match_id and a.season=b.season_year and a.bowler=b.bowler
where b.wkts>1 
order by a.season asc , sr asc;"""

spark.sql(f"""
    CREATE OR REPLACE TABLE Best_Bowling_Strike_Rate_Innings
    USING DELTA
    PARTITIONED BY (season_year)
    LOCATION "{gold_path}/Best_Bowling_Strike_Rate_Innings"
    AS {Best_Bowling_Strike_Rate_Innings}
""")

Most_Runs_Conceded_Innings="""
select 
a.season as season_year,
a.bowler as player_id,
b.player_name,
b.player_team,
b.runs,
b.ov,
b.wkts,
b.sr,
b.AGNAIST, 
b.venue,
b.match_date
from bowler_played_matches a
left join 
Best_Bowling_Figures_innings_stg b on a.match_id=b.match_id and a.season=b.season_year and a.bowler=b.bowler
order by a.season asc , runs desc"""

spark.sql(f"""
    CREATE OR REPLACE TABLE Most_Runs_Conceded_Innings
    USING DELTA
    PARTITIONED BY (season_year)
    LOCATION "{gold_path}/Most_Runs_Conceded_Innings"
    AS {Most_Runs_Conceded_Innings}
""")
	
##hatrick

bowler_over_order_wicket_cnt="""
select 
a.season,
a.bowler, 
a.match_id,
a.over_id,
is_wicket_taken,
over_wicket_cnt,
row_number() over(partition by a.season,a.match_id,a.bowler order by a.over_id asc) as over_order_num
from 
(SELECT 
season,bowler, match_id,over_id,max(bowler_wicket)as is_wicket_taken,sum(bowler_wicket) as over_wicket_cnt
from ball_by_ball_stats 
group by 1,2,3,4) a"""

spark.sql(bowler_over_order_wicket_cnt).createOrReplaceTempView("bowler_over_order_wicket_cnt")

bowler_current_over_next_over="""
select 
a.season,a.bowler,a.match_id,
a.over_id as current_over ,
a.is_wicket_taken as current_over_is_wicket_taken,
a.over_order_num as current_over_over_order_num,
a.over_wicket_cnt as current_over_over_wicket_cnt,
b.over_id as next_over ,
b.is_wicket_taken as next_over_is_wicket_taken,
b.over_order_num as next_over_over_order_num,
b.over_wicket_cnt as next_over_over_wicket_cnt
from  bowler_over_order_wicket_cnt a 
join 
bowler_over_order_wicket_cnt b on a.season=b.season and a.match_id=b.match_id 
                              and a.bowler=b.bowler and a.over_order_num+1=b.over_order_num"""

spark.sql(bowler_current_over_next_over).createOrReplaceTempView("bowler_current_over_next_over")

bowler_cont_wick_taken_overs="""
select 
a.*,
b.player_name 
from bowler_current_over_next_over  a
join players_data b on a.bowler=b.player_id
where  
(current_over_over_wicket_cnt=0 and next_over_over_wicket_cnt>=3) or
(current_over_over_wicket_cnt>=3 and next_over_over_wicket_cnt=0) or
(current_over_over_wicket_cnt>=1 and next_over_over_wicket_cnt>=2) or
(current_over_over_wicket_cnt>=2 and next_over_over_wicket_cnt>=1)"""

spark.sql(bowler_cont_wick_taken_overs).createOrReplaceTempView("bowler_cont_wick_taken_overs")

wicket_taken_balls="""
select  
season,
bowler,
match_id,
over_id,ball_id
from ball_by_ball_stats where  bowler_wicket=1"""

spark.sql(wicket_taken_balls).createOrReplaceTempView("wicket_taken_balls")

overs_with_contunious_wickets="""
select a.*,
row_number() over(partition by a.season,a.match_id,a.bowler order by a.over_id,a.ball_id) as rnk
from 
(select 
a.season,a.bowler,a.match_id,a.current_over as over_id ,b.ball_id
from bowler_cont_wick_taken_overs a
left join 
wicket_taken_balls b on a.season=b.season and a.bowler=b.bowler and a.match_id=b.match_id 
and a.current_over=b.over_id
union 
select 
a.season,a.bowler,a.match_id,a.next_over as over_id ,b.ball_id
from bowler_cont_wick_taken_overs a
left join 
wicket_taken_balls b on a.season=b.season and a.bowler=b.bowler and a.match_id=b.match_id 
and a.next_over=b.over_id) a"""

spark.sql(overs_with_contunious_wickets).createOrReplaceTempView("overs_with_contunious_wickets")


hatrick="""
select  
a.season,a.bowler,a.match_id,a.over_id as over_id_1,
a.ball_id as ball_id_1,b.over_id as  over_id_2 ,b.ball_id as  ball_id_2,
c.over_id as over_id_3,c.ball_id as ball_id_3
from overs_with_contunious_wickets a ,
overs_with_contunious_wickets b,
overs_with_contunious_wickets c 
where a.season=b.season and a.match_id=b.match_id and a.bowler=b.bowler and a.rnk+1=b.rnk
and b.season=c.season and b.match_id=c.match_id and b.bowler=c.bowler and b.rnk+1=c.rnk"""

spark.sql(hatrick).createOrReplaceTempView("hatrick")


hatricks="""
select 
hatrick_count.season as season_year,
hatrick_count.bowler as player_id,
pc.player_name,
pc.player_team,
hatrick_count.HAT_TRICKS,
pc.mat,
pc.inns,
pc.ov,
pc.runs,
pc.wkts,
pc.avg,
pc.sr,
pc.`4W`,
pc.`5W`
from
 (select a.season,a.bowler,count(*) as HAT_TRICKS from 
 (select season,bowler,b.player_name,
 case when over_id_1=over_id_2 and over_id_2=over_id_3
           and (ball_id_1+1=ball_id_2 and ball_id_2+1=ball_id_3) then 'YES' --Hartick in same Over
	  when (ball_id_1=5 and ball_id_2=6 and over_id_1=over_id_2  and ball_id_3 =1) then 'YES'
	  when (ball_id_1=6 and ball_id_2=1 and ball_id_3 =2 and over_id_2=over_id_3) then 'YES'
	  when (ball_id_1=7 and ball_id_2=1 and ball_id_3 =2  and over_id_2=over_id_3) then 'YES'
      when (ball_id_1=6 and ball_id_2=7  and over_id_1=over_id_2 and ball_id_3 =1) then 'YES'
   else 'NO' end as hatrick
 from hatrick a
 join 
  Players_data b on a.bowler=b.player_id
 ) a where hatrick='YES'
 group by 1,2
) hatrick_count
join PURPLE_CAP PC on hatrick_count.season=pc.season_year and hatrick_count.bowler=pc.player_id
 order by season asc;"""


spark.sql(f"""
    CREATE OR REPLACE TABLE hatricks
    USING DELTA
    PARTITIONED BY (season_year)
    LOCATION "{gold_path}/hatricks"
    AS {hatricks}
""")



Out[7]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
%fs ls /FileStore/ipl_data/gold/bowlers

path,name,size,modificationTime
dbfs:/FileStore/ipl_data/gold/bowlers/Best_Bowling_Economy_Innings/,Best_Bowling_Economy_Innings/,0,0
dbfs:/FileStore/ipl_data/gold/bowlers/Best_Bowling_Figures_innings/,Best_Bowling_Figures_innings/,0,0
dbfs:/FileStore/ipl_data/gold/bowlers/Best_Bowling_Strike_Rate_Innings/,Best_Bowling_Strike_Rate_Innings/,0,0
dbfs:/FileStore/ipl_data/gold/bowlers/Most_Dot_Balls_Innings/,Most_Dot_Balls_Innings/,0,0
dbfs:/FileStore/ipl_data/gold/bowlers/Most_Maidens/,Most_Maidens/,0,0
dbfs:/FileStore/ipl_data/gold/bowlers/Most_Runs_Conceded_Innings/,Most_Runs_Conceded_Innings/,0,0
dbfs:/FileStore/ipl_data/gold/bowlers/best_bowling_average/,best_bowling_average/,0,0
dbfs:/FileStore/ipl_data/gold/bowlers/best_bowling_economy/,best_bowling_economy/,0,0
dbfs:/FileStore/ipl_data/gold/bowlers/best_bowling_strike_rate/,best_bowling_strike_rate/,0,0
dbfs:/FileStore/ipl_data/gold/bowlers/hatricks/,hatricks/,0,0


Merge All DataFrames to get final output.